In [ ]:
from qcrypto.agents import Agent
from qcrypto.qstates import QstateEnt, QstateUnEnt
from qcrypto.gates import H_gate
import numpy as np

numqubits = 20
public_qstate = QstateUnEnt(init_method="random", num_qubits=numqubits)
def Agent_profile():
    Alice = Agent()
    Alice.set_qstate(qstate=public_qstate,qstate_type='public')
    Alice_base_choice = np.random.choice([True, False], size=(numqubits))
    Alice_to_gate = np.nonzero(Alice_base_choice)[0]
    Alice.apply_gate(qstate_type="public", gate=H_gate, qubit_idx=Alice_to_gate)
    Alice.get_key(qstate_type="public")
    Alice.apply_gate(qstate_type="public", gate=H_gate, qubit_idx=Alice_to_gate)

In [ ]:
%load_ext line_profiler

In [ ]:
# Running in units of µs
%lprun -u 0.000_001 -T Agent_profile.txt -f Agent_profile Agent_profile()

In [ ]:
%lprun -u 0.000_001 -T qstate_measure_all.txt -f public_qstate.measure_all public_qstate.measure_all() 

In [ ]:
%lprun -u 0.000_001 -T qstate_measure.txt -f public_qstate.measure [public_qstate.measure(i) for i in range(numqubits) ]

The issue seems to be Numpy's `rng.choice` method

In [ ]:
rng = np.random.default_rng()

In [ ]:
%%timeit
rng.choice([0,1],p = [.02,.98])

There's not much more to improve here now. We would need to either make a faster implementation of NumPy's `rng.choice` method or try Numba.
However Numba does not support type annotations yet so will leave it as is.